In [1]:
from collections import deque

with open("inputs/10.txt", 'r') as fh:
    sketch = [list(line.strip()) for line in fh.readlines() if line.strip()]

In [2]:
H = len(sketch)
W = len(sketch[0])

potential = {
    "|": [(-1, 0), (1, 0)],
    "-": [(0, -1), (0, 1)],
    "L": [(-1, 0), (0, 1)],
    "J": [(-1, 0), (0, -1)],
    "7": [(0, -1), (1, 0)],
    "F": [(0, 1), (1, 0)],
}

def get_neighbors(i, j):
    neighbors = []
    for oi, oj in potential[sketch[i][j]]:
        i_ = i + oi
        j_ = j + oj
        if i_ >= 0 and i_ < H and j_ >= 0 and j_ < W:
            neighbors.append((i_, j_))
    return neighbors

def find(arr2d, val):
    for i in range(H):
        for j in range(W):
            if arr2d[i][j] == val:
                return (i, j)

def argmax(arr2d):
    mi = 0
    mj = 0
    for i in range(H):
        for j in range(W):
            if arr2d[i][j] is not None and (arr2d[mi][mj] is None or arr2d[i][j] > arr2d[mi][mj]):
                mi, mj = i, j
    return (mi, mj)

In [3]:
# Find S
i0, j0 = find(sketch, "S")

In [4]:
# Replace S according to its neighbors
west = j0 > 0 and sketch[i0][j0-1] in "-LF"
east = j0 < W-1 and sketch[i0][j0+1] in "-7J"
north = i0 > 0 and sketch[i0-1][j0] in "|F7"
south = i0 < H-1 and sketch[i0+1][j0] in "|JL"
if north and south:
    sketch[i0][j0] = "|"
if east and west:
    sketch[i0][j0] = "-"
elif north and east:
    sketch[i0][j0] = "L"
elif north and west:
    sketch[i0][j0] = "J"
elif south and west:
    sketch[i0][j0] = "7"
elif south and east:
    sketch[i0][j0] = "F"

In [5]:
# Part 1: Dijkstra
dist = [[None] * W for _ in range(H)]
dist[i0][j0] = 0
queue = deque()
queue.append((i0, j0))
while queue:
    i, j = queue.popleft()
    d = dist[i][j]
    for i_, j_ in get_neighbors(i, j):
        if dist[i_][j_] is None:
            dist[i_][j_] = d + 1
            queue.append((i_, j_))
mi, mj = argmax(dist)
print(dist[mi][mj])

6831


In [6]:
# Part 2
# Replace all non-loop tiles with "."
for i in range(H):
    for j in range(W):
        if dist[i][j] is None:
            sketch[i][j] = "."

In [7]:
# Create a larger sketch from the small sketch, fill the gaps, mark the outside
HL = 2 * H + 1
WL = 2 * W + 1
bigsketch = [["."]*WL for _ in range(HL)]
for i in range(H):
    for j in range(W):
        bigsketch[2*i+1][2*j+1] = sketch[i][j]
for i in range(1, HL-1):
    for j in range(1, WL-1):
        if (i + j) % 2 == 1:
            west = bigsketch[i][j-1] in "-LF"
            east = bigsketch[i][j+1] in "-7J"
            north = bigsketch[i-1][j] in "|F7"
            south = bigsketch[i+1][j] in "|JL"
            if (north and south) or (west and east):
                bigsketch[i][j] = "x"
for i in range(HL):
    bigsketch[i][0] = "O"
    bigsketch[i][-1] = "O"
for j in range(WL):
    bigsketch[0][j] = "O"
    bigsketch[-1][j] = "O"

# Start Dijkstra from the Os
queue = deque([(i, j) for i in range(HL) for j in range(WL) if bigsketch[i][j] == "O"])
while queue:
    i, j = queue.popleft()
    for i_, j_ in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]:
        if i_ >= 0 and i_ < HL and j_ >= 0 and j_ < WL and bigsketch[i_][j_] == ".":
            bigsketch[i_][j_] = "O"
            queue.append((i_, j_))

# Count inside tiles in small sketch
for i in range(H):
    for j in range(W):
        sketch[i][j] = bigsketch[2*i+1][2*j+1]
print(sum(1 if sketch[i][j] == "." else 0 for i in range(H) for j in range(W)))


305
